# 18장 몬테 카를로 시뮬레이션

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/python4daml/blob/main/18장.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 18.1 파스칼의 문제

In [2]:
import random 
import numpy as np 

In [3]:
def roll_die(): 
    return random.choice([1,2,3,4,5,6]) 

def check_pascal(num_trials): 
    """num_trials은 int > 0라고 가정합니다.
       이길 확률을 출력합니다""" 
    num_wins = 0 
    for i in range(num_trials): 
        for j in range(24): 
            d1 = roll_die() 
            d2 = roll_die() 
            if d1 == 6 and d2 == 6: 
                num_wins += 1 
                break 
    print('이길 확률 =', num_wins/num_trials) 

In [4]:
check_pascal(1000000)

이길 확률 = 0.491649


## 18.2 Pass 또는 Don’t pass?

그림 18-2 `Craps_game` 클래스

In [5]:
class Craps_game(object): 
    def __init__(self): 
        self.pass_wins, self.pass_losses = 0, 0 
        self.dp_wins, self.dp_losses, self.dp_pushes = 0, 0, 0 

    def play_hand(self): 
        throw = roll_die() + roll_die() 
        if throw == 7 or throw == 11: 
            self.pass_wins += 1 
            self.dp_losses += 1 
        elif throw == 2 or throw == 3 or throw == 12: 
            self.pass_losses += 1 
            if throw == 12: 
                self.dp_pushes += 1 
            else: 
                self.dp_wins += 1 
        else: 
            point = throw 
            while True: 
                throw = roll_die() + roll_die() 
                if throw == point: 
                    self.pass_wins += 1 
                    self.dp_losses += 1 
                    break 
                elif throw == 7: 
                    self.pass_losses += 1 
                    self.dp_wins += 1 
                    break 

    def pass_results(self): 
        return (self.pass_wins, self.pass_losses) 

    def dp_results(self): 
        return (self.dp_wins, self.dp_losses, self.dp_pushes) 

그림 18-3 크랩스 게임 시뮬레이션하기

In [6]:
def craps_sim(hands_per_game, num_games): 
    """hands_per_game과 num_games는 ints > 0이라고 가정합니다.
       hands_per_game번 핸드를 반복하는 num_games번 게임을 플레이하고 결과를 출력합니다"""
    games = [] 

    #num_games번 게임을 플레이합니다
    for t in range(num_games): 
        c = Craps_game() 
        for i in range(hands_per_game): 
            c.play_hand() 
        games.append(c) 

    #각 게임에 대한 통계를 생성합니다
    p_ROI_per_game, dp_ROI_per_game = [], [] 
    for g in games: 
        wins, losses = g.pass_results() 
        p_ROI_per_game.append((wins - losses)/float(hands_per_game)) 
        wins, losses, pushes = g.dp_results() 
        dp_ROI_per_game.append((wins - losses)/float(hands_per_game)) 

    #요약 통계를 만들어 출력합니다
    mean_ROI = str(round((100*sum(p_ROI_per_game)/num_games), 4)) + '%' 
    sigma = str(round(100*np.std(p_ROI_per_game), 4)) + '%' 
    print('패스:', '평균 ROI =', mean_ROI, '표준 편차 =', sigma) 
    mean_ROI = str(round((100*sum(dp_ROI_per_game)/num_games), 4)) +'%' 
    sigma = str(round(100*np.std(dp_ROI_per_game), 4)) + '%' 
    print('돈 패스:','평균 ROI =', mean_ROI, '표준 편차 =', sigma) 

In [7]:
craps_sim(20, 10)

패스: 평균 ROI = -16.0% 표준 편차 = 18.5472%
돈 패스: 평균 ROI = 14.0% 표준 편차 = 17.72%


In [8]:
craps_sim(1000000, 10)

패스: 평균 ROI = -1.4183% 표준 편차 = 0.088%
돈 패스: 평균 ROI = -1.3642% 표준 편차 = 0.0878%


In [9]:
craps_sim(20, 1000000)

패스: 평균 ROI = -1.4188% 표준 편차 = 22.3804%
돈 패스: 평균 ROI = -1.3633% 표준 편차 = 22.0663%


In [10]:
def roll_die(): 
    return random.choice([1,1,2,3,3,4,4,5,5,5,6,6]) 

In [11]:
craps_sim(1000000, 10)

패스: 평균 ROI = 6.7131% 표준 편차 = 0.0804%
돈 패스: 평균 ROI = -9.4902% 표준 편차 = 0.0744%


**손가락 운동**

In [18]:
def roll_die(): 
    return random.choice([1,2,3,4,5,6]) 

class Craps_game_big6(object): 
    def __init__(self): 
        self.b6_wins, self.b6_losses = 0, 0

    def play_hand(self): 
        throw = roll_die() + roll_die() 
        if throw == 6:
            self.b6_wins += 1
        elif throw == 7:
            self.b6_losses += 1
        else: 
            point = throw 
            while True: 
                throw = roll_die() + roll_die() 
                if throw == 6:
                    self.b6_wins += 1
                elif throw == point: 
                    break 
                elif throw == 7: 
                    self.b6_losses += 1
                    break 

    def b6_results(self):
        return (self.b6_wins, self.b6_losses)

def craps_sim_b6(hands_per_game, num_games): 
    """hands_per_game과 num_games는 ints > 0이라고 가정합니다.
       hands_per_game번 핸드를 반복하는 num_games번 게임을 플레이하고 결과를 출력합니다"""
    games = [] 

    #num_games번 게임을 플레이합니다
    for t in range(num_games): 
        c = Craps_game_big6() 
        for i in range(hands_per_game): 
            c.play_hand() 
        games.append(c) 

    #각 게임에 대한 통계를 생성합니다
    b6_cost_per_hour = [] 
    for g in games: 
        wins, losses = g.b6_results() 
        b6_cost_per_hour.append((wins - losses)*5/(float(hands_per_game)/30)) 

    #요약 통계를 만들어 출력합니다
    mean_cost = str(round(sum(b6_cost_per_hour)/num_games, 4)) + '달러' 
    sigma = str(round(np.std(b6_cost_per_hour), 4)) + '달러' 
    print('빅 6:', '시간당 평균 비용 =', mean_cost, '표준 편차 =', sigma)  

craps_sim_b6(1000000, 10)

빅 6: 시간당 평균 비용 = -15.5022달러 표준 편차 = 0.1291달러


그림 18-4 테이블 룩업을 사용해 성능 향상하기

In [19]:
def play_hand(self): 
    #빠른 버전의 play_hand 구현
    points_dict = {4:1/3, 5:2/5, 6:5/11, 8:5/11, 9:2/5, 10:1/3} 
    throw = roll_die() + roll_die() 
    if throw == 7 or throw == 11: 
        self.pass_wins += 1 
        self.dp_losses += 1 
    elif throw == 2 or throw == 3 or throw == 12: 
        self.pass_losses += 1 
        if throw == 12: 
            self.dp_pushes += 1 
        else: 
            self.dp_wins += 1 
    else: 
        if random.random() <= points_dict[throw]: # 7전에 포인트가 나옴
            self.pass_wins += 1 
            self.dp_losses += 1 
        else: # 포인트 전에 7이 나옴
            self.pass_losses += 1 
            self.dp_wins += 1 

## 18.4 𝜋 찾기

In [20]:
def throw_needles(num_needles): 
    in_circle = 0 
    for Needles in range(1, num_needles + 1): 
        x = random.random() 
        y = random.random() 
        if (x*x + y*y)**0.5 <= 1: 
            in_circle += 1 
    #1사분면 안에 있는 바늘만 세므로 4를 곱합니다
    return 4*(in_circle/num_needles) 

def get_est(num_needles, num_trials): 
    estimates = [] 
    for t in range(num_trials): 
        pi_guess = throw_needles(num_needles) 
        estimates.append(pi_guess) 
    std_dev = np.std(estimates) 
    cur_est = sum(estimates)/len(estimates) 
    print('𝜋 =', str(round(cur_est, 5)) + ',', 
          '표준 편차 =', str(round(std_dev, 5)) + ',', 
          '바늘 개수 =', num_needles) 
    return (cur_est, std_dev) 

def est_pi(precision, num_trials): 
    num_needles = 1000 
    std_dev = precision 
    while std_dev > precision/1.96: 
        cur_est, std_dev = get_est(num_needles, num_trials) 
        num_needles *= 2 
    return cur_est 

In [21]:
est_pi(0.01, 100)

𝜋 = 3.13376, 표준 편차 = 0.05295, 바늘 개수 = 1000
𝜋 = 3.13958, 표준 편차 = 0.03457, 바늘 개수 = 2000
𝜋 = 3.13715, 표준 편차 = 0.02393, 바늘 개수 = 4000
𝜋 = 3.13959, 표준 편차 = 0.01665, 바늘 개수 = 8000
𝜋 = 3.14189, 표준 편차 = 0.01172, 바늘 개수 = 16000
𝜋 = 3.14267, 표준 편차 = 0.00935, 바늘 개수 = 32000
𝜋 = 3.14134, 표준 편차 = 0.00634, 바늘 개수 = 64000
𝜋 = 3.14215, 표준 편차 = 0.00513, 바늘 개수 = 128000
𝜋 = 3.14117, 표준 편차 = 0.00313, 바늘 개수 = 256000


3.1411725

In [23]:
def throw_needles(num_needles): 
    in_circle = 0 
    for Needles in range(1, num_needles + 1): 
        x = random.random() 
        y = random.random() 
        if (x*x + y*y)**0.5 <= 1: 
            in_circle += 1 
    #1사분면 안에 있는 바늘만 세므로 4를 곱합니다
    return 2*(in_circle/num_needles) 

est_pi(0.01, 100)

𝜋 = 1.57292, 표준 편차 = 0.02748, 바늘 개수 = 1000
𝜋 = 1.57332, 표준 편차 = 0.01727, 바늘 개수 = 2000
𝜋 = 1.57106, 표준 편차 = 0.01342, 바늘 개수 = 4000
𝜋 = 1.57317, 표준 편차 = 0.00908, 바늘 개수 = 8000
𝜋 = 1.57138, 표준 편차 = 0.00652, 바늘 개수 = 16000
𝜋 = 1.5719, 표준 편차 = 0.00428, 바늘 개수 = 32000


1.5719043750000008